In [1]:
import os
from mistralai import Mistral
from dotenv import load_dotenv
import pandas as pd
load_dotenv()
import time
from pymongo import MongoClient
import json
import asyncio


In [2]:
api_key = os.environ["mistral_key"]
gen_ai_key = os.environ["gen_ai_key"]
agent_id = os.environ["agent_id"]
db_url = os.environ["DB_URL"]
model = "open-mistral-nemo"
client = Mistral(api_key=api_key)
mongo_client = MongoClient(db_url)
db  = mongo_client.reddit
post_db = db['post_new']

In [3]:
rec = post_db.find()
rec = list(rec)
print(len(rec)) 

10001


In [4]:
df = pd.DataFrame(rec)

In [5]:
df = df.drop_duplicates(subset=['title'])

In [6]:
len(df)

875

In [9]:
new_post_db = db['post_new_final']
new_post_db.insert_many(df.to_dict('records'))

InsertManyResult([ObjectId('67558db3eeb7770cf0a37c63'), ObjectId('67558db3eeb7770cf0a37c64'), ObjectId('67558db3eeb7770cf0a37c65'), ObjectId('67558db3eeb7770cf0a37c66'), ObjectId('67558db3eeb7770cf0a37c67'), ObjectId('67558db3eeb7770cf0a37c68'), ObjectId('67558db3eeb7770cf0a37c69'), ObjectId('67558db3eeb7770cf0a37c6a'), ObjectId('67558db3eeb7770cf0a37c6b'), ObjectId('67558db3eeb7770cf0a37c6c'), ObjectId('67558db3eeb7770cf0a37c6d'), ObjectId('67558db3eeb7770cf0a37c6e'), ObjectId('67558db3eeb7770cf0a37c6f'), ObjectId('67558db3eeb7770cf0a37c70'), ObjectId('67558db3eeb7770cf0a37c71'), ObjectId('67558db3eeb7770cf0a37c72'), ObjectId('67558db3eeb7770cf0a37c73'), ObjectId('67558db3eeb7770cf0a37c74'), ObjectId('67558db3eeb7770cf0a37c75'), ObjectId('67558db3eeb7770cf0a37c76'), ObjectId('67558db3eeb7770cf0a37c77'), ObjectId('67558db3eeb7770cf0a37c78'), ObjectId('67558db3eeb7770cf0a37c79'), ObjectId('67558db3eeb7770cf0a37c7a'), ObjectId('67558db3eeb7770cf0a37c7b'), ObjectId('67558db3eeb7770cf0a37c

In [10]:
comments_db = db['comments_db']
comms = comments_db.find()
comms = list(comms)
print(len(comms))

30


In [11]:
df2 = pd.DataFrame(comms)
df2 = df2.drop_duplicates(subset=['parent_id'])
print(len(df2))

3


In [8]:
df.to_csv('final.csv', index=False)

In [20]:
def update_post(post):
    try:
        post_db.update_one({'id':post['id']},{'$set':{'response':post['response']}},upsert=True)
        # print(f'Post updated [{post["id"]}]')
    except Exception as e:
        print(e)

In [23]:
count = 0
for i in rec[:5]:
    chat_response = client.agents.complete(
        agent_id=agent_id,
        messages=[
            {
                "role": "user",
                "content": f"post_id:{i['id']},title: {i['title']},body: {i['selftext']}" ,
            },
        ],
    )
    response = eval(chat_response.choices[0].message.content)
    i['response'] = response['flag']
    update_post(i) 
    count += 1
    print(f'{count}. title:{i["title"]},response:{response["flag"]}')

1. title:Do you think this startup is worth continuing?,response:question
2. title:Finally launched my LinkedIn AI chatbot!,response:experience
3. title:Finally!! I launched it after months ,response:experience
4. title:Struggling with Low Conversions? Here’s How Startup-Specific Copy Can Help,response:question
5. title:Im so impatient and its killing me,response:question


In [12]:
import praw

# Reddit API credentials

# Reddit API credentials
client_id       = "13ebR3osBuMhCFhRN8NtBw"
client_secret   = "4KvI1iGAGrwhtt_2qvkqOOW09c6vCQ"
user_agent      = "extractor by u/VishwanathAS"

# Initialize the Reddit instance
reddit = praw.Reddit(
    client_id=client_id,
    client_secret=client_secret,
    user_agent=user_agent
)

In [13]:
def scrape_reddit_comments(post_id):
    """
    Scrape comments from a Reddit post using its post ID.
    Args:
        post_id (str): The Reddit post ID.
    Returns:
        list: A list of comments from the post.
    """
    try:
        # Get the submission object using the post ID
        submission = reddit.submission(id=post_id)
        submission.comments.replace_more(limit=25)  # Remove "more comments"
        
        comments = []
        for comment in submission.comments.list():
            comments.append(comment.body)
        
        return comments

    except Exception as e:
        print(f"An error occurred: {e}")
        return []

In [ ]:
post_id = df['id'][0]  # Replace with the actual post ID
    
for post_id in df['id'][0:5]:
    comments = scrape_reddit_comments(post_id)

if comments:
    print("Comments scraped successfully:")
    for i, comment in enumerate(comments[:10], 1):  # Display first 10 comments
        print(f"{i}. {comment}\n")
else:
    print("No comments found or an error occurred.")

In [25]:
df['id'][0]

'1h9gtbi'

In [ ]:
import requests

url = "http://localhost:5678/webhook/post_comm"
test_url="http://localhost:5678/webhook-test/post_comm"
count = 0
for i in range(len(df)):
    count += 1
    if df['num_comments'][i] <= 0:
        print(f'{i} has no comments')
        continue
    body = {
        "post_id": df['id'][i],
        }
    resp = requests.post(url, body)
    print(f'{count}. {resp.text}')
    time.sleep(2)

In [7]:
comments_db = db['comment_db']
comms = comments_db.find()
comms = list(comms)
print(len(comms))
df2 = pd.DataFrame(comms)
df2 = df2.drop_duplicates(subset=['parent_id'])
len(df2)
comm__post_ids = df2['parent_id'].tolist()
comm__post_ids = [i.split('_')[1] for i in comm__post_ids]
print(comm__post_ids)


remaining_post_ids = list(set(df['id'].tolist()) - set(comm__post_ids))
print(len(remaining_post_ids))

1370
['1h9gtbi', '1h9e0an', '1h9cua9', '1h9achm', '1h9ac56', '1h980yx', '1h96sie', '1h96sbl', '1h96d8c', '1h965rt', '1h94vta', '1h94egx', '1h9347e', '1h92zk6', '1h91q7g', '1h913zy', '1h90nxb', '1h8wpup', '1h8wajl', '1h8u5d3', '1h8tnov', '1h8tgei', '1h8ss1k', '1h8r9fp', '1h8qq7b', '1h8pt5k', '1h8prw9', '1h8mm84', '1h8kbbb', '1h8jv31', '1h8iw8c', '1h8ivih', '1h8iqv1', '1h8h50y', '1h8glkz', '1h8g50t', '1h8cue9', '1h8cmw5', '1h8b5hh', '1h8b1ik', '1h8b03i', '1h8aj0a', '1h892yq', '1h88f75', '1h87kt3', '1h87a6u', '1h848tt', '1h829qs', '1h81uv4', '1h813eu', '1h80z8i', '1h7z0oa', '1h7x1in', '1h7vtze', '1h7ups3', '1h7sz7d', '1h7swwh', '1h7qtzb', '1h7pwsc', '1h7o9vp', '1h7naik', '1h7na49', '1h7lbr2', '1h7kz6j', '1h7kb2v', '1h7k5e0', '1h7jhe7', '1h7h4re', '1h7g55b', '1h7f9zv', '1h7eo7c', '1h7dvkr', '1h7aic5', '1h7a4sa', '1h74irl', '1h745jt', '1h71115', '1h6zf4y', '1h6yt0d', '1h6xbny', '1h6vx4p', '1h6vhar', '1h6rslq', '1h6rank', '1h6r9vv', '1h6pm9q', '1h6nfv4', '1h6layg', '1h6k53z', '1h6hqyr', '1gx

In [12]:
post_id ='1h9gtbi'
res = df[df['id'] == post_id]['num_comments'].values[0]
res

np.int64(6)

In [8]:

import requests
url = "http://localhost:5678/webhook/post_comm"
count = 0
for i in remaining_post_ids:
    if df[df['id'] == i]['num_comments'].values[0] <= 0:
        print(f'{i} has no comments')
        continue
    body = {
        "post_id": i,
        }
    resp = requests.post(url, body)
    print(f'{i}. {resp.text}')
    time.sleep(2)

1h5zslp has no comments
1glncg3. {"comm_id":"lvvjo2j","body":"Hey you can check grant page of techuplabs. Page :  techuplabs\\[dot\\]com/startup-business-grants","parent_id":"t3_1glncg3","_id":"675c8225a25f65c1ac0b3709","id":"675c8225a25f65c1ac0b3709"}
1h3put5. {"comm_id":"lzsoq5r","body":"Man I have been just thinking about this today and you said it. \n\nI have been focusing so much on the “lady in the red dress” lately with beautiful dashboards, building private solution instead of using paid APIs and all that tech stacks adds up. \n\nThen it hit me to just build SOMETHING and ship and be in a position to add one feature a week that’s it instead of cramming everything together in an MVP.","parent_id":"t3_1h3put5","_id":"675c8228a25f65c1ac0b370d","id":"675c8228a25f65c1ac0b370d"}
1gq7y84. {"comm_id":"lwwbjth","body":"We just finished raising a pre-seed so I can share a bit about how we approached it. This is my first time as a founder so I’m by no means an expert. \n\nFirst off, a lot

: 

: 

: 